# Assignment 2
# Matthew Wirasakti
## Problem 1

In [24]:
# Make a prediction with weights
def predict(row, weights):
    activation = weights[0] # bias
    for i in range(len(row) - 1):
        activation += weights[i + 1] * row[i]
    return 1.0 if activation >= 0.0 else 0.0

In [25]:
# test predictions
dataset=[[2.7810836,2.550537003,0],[1.465489372,2.362125076,0],[3.396561688,4.400293529,0],
         [1.38807019,1.850220317,0],[3.06407232,3.005305973,0],[7.627531214,2.759262235,1],
         [5.332441248,2.088626775,1],[6.922596716,1.77106367,1],[8.675418651,-0.242068655,1],
         [7.673756466,3.508563011,1]]
weights=[-0.1,0.20653640140000007,-0.23418117710000003]# I copied trained weights
for row in dataset:
    prediction = predict(row, weights)
    print("Expected=%d, Predicted=%d" % (row[-1], prediction))

Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=0, Predicted=0
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1
Expected=1, Predicted=1


In [26]:
# Estimate Perceptron weights using stochastic gradient descent
def train_weights(train, l_rate, n_epoch):
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            sum_error += error**2
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row) - 1):
                weights[i+1] = weights[i + 1] + l_rate * error * row[i]
        print('>epoch = %d, lrate = %.3f, error = %.3f' % (epoch, l_rate, sum_error))
    return weights

In [27]:
l_rate = 0.1
n_epoch = 5
weights = train_weights(dataset, l_rate, n_epoch)
print(weights)

>epoch = 0, lrate = 0.100, error = 2.000
>epoch = 1, lrate = 0.100, error = 1.000
>epoch = 2, lrate = 0.100, error = 0.000
>epoch = 3, lrate = 0.100, error = 0.000
>epoch = 4, lrate = 0.100, error = 0.000
[-0.1, 0.20653640140000007, -0.23418117710000003]


In [28]:
# Perceptron Algorithm on the Sonar Dataset
from random import seed
from random import randrange
from csv import reader
from sklearn.metrics import classification_report
import numpy as np

In [29]:
# Load a CSV file
def load_csv(filename):
    dataset = list()
    with open(filename, 'r') as file:
        csv_reader = reader(file)
        for row in csv_reader:
            if not row:
                continue
            dataset.append(row)
    return dataset

In [30]:
# Convert string column to float
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

In [31]:
# Convert string column to integer
def str_column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

In [32]:
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [33]:
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

In [34]:
# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1]for row in fold]
        accuracy = accuracy_metric(actual, predicted)
        scores.append(accuracy)
        print(classification_report(actual, predicted, zero_division = 0))
    return scores

In [35]:
# Perceptron Algorithm With Stochastic Gradient Descent
def perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return(predictions)

In [36]:
# Test the Perceptron algorithm on the sonar dataset
seed(0)
# load and prepare data
filename = 'sonar.all-data.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0]) - 1):
    str_column_to_float(dataset, i)
# convert string class to integers
str_column_to_int(dataset, len(dataset[0]) - 1)

{'R': 0, 'M': 1}

In [37]:
# evaluate algorithm
# n_folds = 3
# l_rate = 0.01
# n_epoch = 500
diff_l_rates = [0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.10]
# diff_l_rates = [0.01, 0.001, 0.0001]
for i in diff_l_rates:
    n_folds = 3
    n_epoch = 500
    print("Learning rate = ", i)
    scores = evaluate_algorithm(dataset, perceptron, n_folds, i, n_epoch)
    print('Scores: %s' % scores)
    print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))
    print()

Learning rate =  0.01
>epoch = 0, lrate = 0.010, error = 72.000
>epoch = 1, lrate = 0.010, error = 44.000
>epoch = 2, lrate = 0.010, error = 40.000
>epoch = 3, lrate = 0.010, error = 49.000
>epoch = 4, lrate = 0.010, error = 46.000
>epoch = 5, lrate = 0.010, error = 40.000
>epoch = 6, lrate = 0.010, error = 45.000
>epoch = 7, lrate = 0.010, error = 41.000
>epoch = 8, lrate = 0.010, error = 37.000
>epoch = 9, lrate = 0.010, error = 34.000
>epoch = 10, lrate = 0.010, error = 32.000
>epoch = 11, lrate = 0.010, error = 32.000
>epoch = 12, lrate = 0.010, error = 37.000
>epoch = 13, lrate = 0.010, error = 40.000
>epoch = 14, lrate = 0.010, error = 31.000
>epoch = 15, lrate = 0.010, error = 40.000
>epoch = 16, lrate = 0.010, error = 38.000
>epoch = 17, lrate = 0.010, error = 31.000
>epoch = 18, lrate = 0.010, error = 34.000
>epoch = 19, lrate = 0.010, error = 32.000
>epoch = 20, lrate = 0.010, error = 34.000
>epoch = 21, lrate = 0.010, error = 34.000
>epoch = 22, lrate = 0.010, error = 31.000

              precision    recall  f1-score   support

           0       0.90      0.29      0.44        31
           1       0.63      0.97      0.76        38

    accuracy                           0.67        69
   macro avg       0.76      0.63      0.60        69
weighted avg       0.75      0.67      0.62        69

>epoch = 0, lrate = 0.010, error = 56.000
>epoch = 1, lrate = 0.010, error = 55.000
>epoch = 2, lrate = 0.010, error = 49.000
>epoch = 3, lrate = 0.010, error = 51.000
>epoch = 4, lrate = 0.010, error = 48.000
>epoch = 5, lrate = 0.010, error = 49.000
>epoch = 6, lrate = 0.010, error = 44.000
>epoch = 7, lrate = 0.010, error = 40.000
>epoch = 8, lrate = 0.010, error = 42.000
>epoch = 9, lrate = 0.010, error = 39.000
>epoch = 10, lrate = 0.010, error = 30.000
>epoch = 11, lrate = 0.010, error = 36.000
>epoch = 12, lrate = 0.010, error = 39.000
>epoch = 13, lrate = 0.010, error = 32.000
>epoch = 14, lrate = 0.010, error = 37.000
>epoch = 15, lrate = 0.010, error = 31

>epoch = 229, lrate = 0.010, error = 18.000
>epoch = 230, lrate = 0.010, error = 13.000
>epoch = 231, lrate = 0.010, error = 25.000
>epoch = 232, lrate = 0.010, error = 16.000
>epoch = 233, lrate = 0.010, error = 20.000
>epoch = 234, lrate = 0.010, error = 15.000
>epoch = 235, lrate = 0.010, error = 19.000
>epoch = 236, lrate = 0.010, error = 16.000
>epoch = 237, lrate = 0.010, error = 22.000
>epoch = 238, lrate = 0.010, error = 24.000
>epoch = 239, lrate = 0.010, error = 22.000
>epoch = 240, lrate = 0.010, error = 19.000
>epoch = 241, lrate = 0.010, error = 23.000
>epoch = 242, lrate = 0.010, error = 16.000
>epoch = 243, lrate = 0.010, error = 18.000
>epoch = 244, lrate = 0.010, error = 26.000
>epoch = 245, lrate = 0.010, error = 17.000
>epoch = 246, lrate = 0.010, error = 21.000
>epoch = 247, lrate = 0.010, error = 19.000
>epoch = 248, lrate = 0.010, error = 18.000
>epoch = 249, lrate = 0.010, error = 22.000
>epoch = 250, lrate = 0.010, error = 14.000
>epoch = 251, lrate = 0.010, err

>epoch = 56, lrate = 0.010, error = 30.000
>epoch = 57, lrate = 0.010, error = 34.000
>epoch = 58, lrate = 0.010, error = 28.000
>epoch = 59, lrate = 0.010, error = 33.000
>epoch = 60, lrate = 0.010, error = 30.000
>epoch = 61, lrate = 0.010, error = 36.000
>epoch = 62, lrate = 0.010, error = 27.000
>epoch = 63, lrate = 0.010, error = 34.000
>epoch = 64, lrate = 0.010, error = 30.000
>epoch = 65, lrate = 0.010, error = 34.000
>epoch = 66, lrate = 0.010, error = 31.000
>epoch = 67, lrate = 0.010, error = 29.000
>epoch = 68, lrate = 0.010, error = 32.000
>epoch = 69, lrate = 0.010, error = 31.000
>epoch = 70, lrate = 0.010, error = 40.000
>epoch = 71, lrate = 0.010, error = 28.000
>epoch = 72, lrate = 0.010, error = 34.000
>epoch = 73, lrate = 0.010, error = 28.000
>epoch = 74, lrate = 0.010, error = 25.000
>epoch = 75, lrate = 0.010, error = 36.000
>epoch = 76, lrate = 0.010, error = 30.000
>epoch = 77, lrate = 0.010, error = 30.000
>epoch = 78, lrate = 0.010, error = 30.000
>epoch = 79

>epoch = 249, lrate = 0.010, error = 22.000
>epoch = 250, lrate = 0.010, error = 19.000
>epoch = 251, lrate = 0.010, error = 23.000
>epoch = 252, lrate = 0.010, error = 22.000
>epoch = 253, lrate = 0.010, error = 20.000
>epoch = 254, lrate = 0.010, error = 23.000
>epoch = 255, lrate = 0.010, error = 24.000
>epoch = 256, lrate = 0.010, error = 23.000
>epoch = 257, lrate = 0.010, error = 23.000
>epoch = 258, lrate = 0.010, error = 18.000
>epoch = 259, lrate = 0.010, error = 25.000
>epoch = 260, lrate = 0.010, error = 24.000
>epoch = 261, lrate = 0.010, error = 22.000
>epoch = 262, lrate = 0.010, error = 22.000
>epoch = 263, lrate = 0.010, error = 26.000
>epoch = 264, lrate = 0.010, error = 22.000
>epoch = 265, lrate = 0.010, error = 30.000
>epoch = 266, lrate = 0.010, error = 20.000
>epoch = 267, lrate = 0.010, error = 26.000
>epoch = 268, lrate = 0.010, error = 22.000
>epoch = 269, lrate = 0.010, error = 26.000
>epoch = 270, lrate = 0.010, error = 16.000
>epoch = 271, lrate = 0.010, err

...

>epoch = 15, lrate = 0.100, error = 35.000
>epoch = 16, lrate = 0.100, error = 36.000
>epoch = 17, lrate = 0.100, error = 36.000
>epoch = 18, lrate = 0.100, error = 34.000
>epoch = 19, lrate = 0.100, error = 34.000
>epoch = 20, lrate = 0.100, error = 35.000
>epoch = 21, lrate = 0.100, error = 34.000
>epoch = 22, lrate = 0.100, error = 33.000
>epoch = 23, lrate = 0.100, error = 34.000
>epoch = 24, lrate = 0.100, error = 36.000
>epoch = 25, lrate = 0.100, error = 33.000
>epoch = 26, lrate = 0.100, error = 32.000
>epoch = 27, lrate = 0.100, error = 27.000
>epoch = 28, lrate = 0.100, error = 34.000
>epoch = 29, lrate = 0.100, error = 30.000
>epoch = 30, lrate = 0.100, error = 37.000
>epoch = 31, lrate = 0.100, error = 30.000
>epoch = 32, lrate = 0.100, error = 27.000
>epoch = 33, lrate = 0.100, error = 34.000
>epoch = 34, lrate = 0.100, error = 28.000
>epoch = 35, lrate = 0.100, error = 33.000
>epoch = 36, lrate = 0.100, error = 28.000
>epoch = 37, lrate = 0.100, error = 22.000
>epoch = 38

>epoch = 228, lrate = 0.100, error = 20.000
>epoch = 229, lrate = 0.100, error = 17.000
>epoch = 230, lrate = 0.100, error = 27.000
>epoch = 231, lrate = 0.100, error = 19.000
>epoch = 232, lrate = 0.100, error = 23.000
>epoch = 233, lrate = 0.100, error = 17.000
>epoch = 234, lrate = 0.100, error = 17.000
>epoch = 235, lrate = 0.100, error = 17.000
>epoch = 236, lrate = 0.100, error = 23.000
>epoch = 237, lrate = 0.100, error = 20.000
>epoch = 238, lrate = 0.100, error = 18.000
>epoch = 239, lrate = 0.100, error = 16.000
>epoch = 240, lrate = 0.100, error = 22.000
>epoch = 241, lrate = 0.100, error = 14.000
>epoch = 242, lrate = 0.100, error = 15.000
>epoch = 243, lrate = 0.100, error = 23.000
>epoch = 244, lrate = 0.100, error = 22.000
>epoch = 245, lrate = 0.100, error = 20.000
>epoch = 246, lrate = 0.100, error = 15.000
>epoch = 247, lrate = 0.100, error = 16.000
>epoch = 248, lrate = 0.100, error = 15.000
>epoch = 249, lrate = 0.100, error = 13.000
>epoch = 250, lrate = 0.100, err

>epoch = 461, lrate = 0.100, error = 14.000
>epoch = 462, lrate = 0.100, error = 26.000
>epoch = 463, lrate = 0.100, error = 20.000
>epoch = 464, lrate = 0.100, error = 16.000
>epoch = 465, lrate = 0.100, error = 17.000
>epoch = 466, lrate = 0.100, error = 15.000
>epoch = 467, lrate = 0.100, error = 12.000
>epoch = 468, lrate = 0.100, error = 17.000
>epoch = 469, lrate = 0.100, error = 11.000
>epoch = 470, lrate = 0.100, error = 16.000
>epoch = 471, lrate = 0.100, error = 20.000
>epoch = 472, lrate = 0.100, error = 18.000
>epoch = 473, lrate = 0.100, error = 14.000
>epoch = 474, lrate = 0.100, error = 12.000
>epoch = 475, lrate = 0.100, error = 24.000
>epoch = 476, lrate = 0.100, error = 18.000
>epoch = 477, lrate = 0.100, error = 21.000
>epoch = 478, lrate = 0.100, error = 24.000
>epoch = 479, lrate = 0.100, error = 8.000
>epoch = 480, lrate = 0.100, error = 13.000
>epoch = 481, lrate = 0.100, error = 8.000
>epoch = 482, lrate = 0.100, error = 12.000
>epoch = 483, lrate = 0.100, error

>epoch = 190, lrate = 0.100, error = 28.000
>epoch = 191, lrate = 0.100, error = 24.000
>epoch = 192, lrate = 0.100, error = 28.000
>epoch = 193, lrate = 0.100, error = 26.000
>epoch = 194, lrate = 0.100, error = 27.000
>epoch = 195, lrate = 0.100, error = 26.000
>epoch = 196, lrate = 0.100, error = 28.000
>epoch = 197, lrate = 0.100, error = 28.000
>epoch = 198, lrate = 0.100, error = 28.000
>epoch = 199, lrate = 0.100, error = 30.000
>epoch = 200, lrate = 0.100, error = 28.000
>epoch = 201, lrate = 0.100, error = 32.000
>epoch = 202, lrate = 0.100, error = 30.000
>epoch = 203, lrate = 0.100, error = 28.000
>epoch = 204, lrate = 0.100, error = 26.000
>epoch = 205, lrate = 0.100, error = 28.000
>epoch = 206, lrate = 0.100, error = 21.000
>epoch = 207, lrate = 0.100, error = 25.000
>epoch = 208, lrate = 0.100, error = 25.000
>epoch = 209, lrate = 0.100, error = 22.000
>epoch = 210, lrate = 0.100, error = 22.000
>epoch = 211, lrate = 0.100, error = 23.000
>epoch = 212, lrate = 0.100, err

>epoch = 401, lrate = 0.100, error = 24.000
>epoch = 402, lrate = 0.100, error = 18.000
>epoch = 403, lrate = 0.100, error = 20.000
>epoch = 404, lrate = 0.100, error = 23.000
>epoch = 405, lrate = 0.100, error = 23.000
>epoch = 406, lrate = 0.100, error = 18.000
>epoch = 407, lrate = 0.100, error = 20.000
>epoch = 408, lrate = 0.100, error = 21.000
>epoch = 409, lrate = 0.100, error = 20.000
>epoch = 410, lrate = 0.100, error = 20.000
>epoch = 411, lrate = 0.100, error = 24.000
>epoch = 412, lrate = 0.100, error = 20.000
>epoch = 413, lrate = 0.100, error = 19.000
>epoch = 414, lrate = 0.100, error = 20.000
>epoch = 415, lrate = 0.100, error = 19.000
>epoch = 416, lrate = 0.100, error = 18.000
>epoch = 417, lrate = 0.100, error = 22.000
>epoch = 418, lrate = 0.100, error = 15.000
>epoch = 419, lrate = 0.100, error = 23.000
>epoch = 420, lrate = 0.100, error = 20.000
>epoch = 421, lrate = 0.100, error = 15.000
>epoch = 422, lrate = 0.100, error = 20.000
>epoch = 423, lrate = 0.100, err

## Problem 2 and Problem 3

In [38]:
#perceptron on 1000+ dimension dataset
import random
seed(1)
dataset1000 = list()
sample = list()
for _ in range(1000):
    sample = [random.random() for _ in range(1000)]
    if sample[0] >= 0:
        sample.append(1)
    elif sample[0] < 0:
        sample.append(0)
    dataset1000.append(sample)

# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))
print()

>epoch = 0, lrate = 0.010, error = 62.000
>epoch = 1, lrate = 0.010, error = 47.000
>epoch = 2, lrate = 0.010, error = 38.000
>epoch = 3, lrate = 0.010, error = 41.000
>epoch = 4, lrate = 0.010, error = 33.000
>epoch = 5, lrate = 0.010, error = 31.000
>epoch = 6, lrate = 0.010, error = 27.000
>epoch = 7, lrate = 0.010, error = 31.000
>epoch = 8, lrate = 0.010, error = 34.000
>epoch = 9, lrate = 0.010, error = 29.000
>epoch = 10, lrate = 0.010, error = 25.000
>epoch = 11, lrate = 0.010, error = 18.000
>epoch = 12, lrate = 0.010, error = 26.000
>epoch = 13, lrate = 0.010, error = 26.000
>epoch = 14, lrate = 0.010, error = 29.000
>epoch = 15, lrate = 0.010, error = 24.000
>epoch = 16, lrate = 0.010, error = 31.000
>epoch = 17, lrate = 0.010, error = 23.000
>epoch = 18, lrate = 0.010, error = 26.000
>epoch = 19, lrate = 0.010, error = 20.000
>epoch = 20, lrate = 0.010, error = 23.000
>epoch = 21, lrate = 0.010, error = 23.000
>epoch = 22, lrate = 0.010, error = 26.000
>epoch = 23, lrate = 

>epoch = 207, lrate = 0.010, error = 10.000
>epoch = 208, lrate = 0.010, error = 8.000
>epoch = 209, lrate = 0.010, error = 4.000
>epoch = 210, lrate = 0.010, error = 2.000
>epoch = 211, lrate = 0.010, error = 2.000
>epoch = 212, lrate = 0.010, error = 4.000
>epoch = 213, lrate = 0.010, error = 8.000
>epoch = 214, lrate = 0.010, error = 8.000
>epoch = 215, lrate = 0.010, error = 8.000
>epoch = 216, lrate = 0.010, error = 6.000
>epoch = 217, lrate = 0.010, error = 8.000
>epoch = 218, lrate = 0.010, error = 4.000
>epoch = 219, lrate = 0.010, error = 6.000
>epoch = 220, lrate = 0.010, error = 4.000
>epoch = 221, lrate = 0.010, error = 8.000
>epoch = 222, lrate = 0.010, error = 10.000
>epoch = 223, lrate = 0.010, error = 8.000
>epoch = 224, lrate = 0.010, error = 4.000
>epoch = 225, lrate = 0.010, error = 2.000
>epoch = 226, lrate = 0.010, error = 4.000
>epoch = 227, lrate = 0.010, error = 8.000
>epoch = 228, lrate = 0.010, error = 2.000
>epoch = 229, lrate = 0.010, error = 2.000
>epoch = 

>epoch = 447, lrate = 0.010, error = 10.000
>epoch = 448, lrate = 0.010, error = 2.000
>epoch = 449, lrate = 0.010, error = 2.000
>epoch = 450, lrate = 0.010, error = 6.000
>epoch = 451, lrate = 0.010, error = 4.000
>epoch = 452, lrate = 0.010, error = 8.000
>epoch = 453, lrate = 0.010, error = 6.000
>epoch = 454, lrate = 0.010, error = 6.000
>epoch = 455, lrate = 0.010, error = 2.000
>epoch = 456, lrate = 0.010, error = 5.000
>epoch = 457, lrate = 0.010, error = 9.000
>epoch = 458, lrate = 0.010, error = 8.000
>epoch = 459, lrate = 0.010, error = 4.000
>epoch = 460, lrate = 0.010, error = 8.000
>epoch = 461, lrate = 0.010, error = 8.000
>epoch = 462, lrate = 0.010, error = 5.000
>epoch = 463, lrate = 0.010, error = 11.000
>epoch = 464, lrate = 0.010, error = 6.000
>epoch = 465, lrate = 0.010, error = 6.000
>epoch = 466, lrate = 0.010, error = 4.000
>epoch = 467, lrate = 0.010, error = 4.000
>epoch = 468, lrate = 0.010, error = 6.000
>epoch = 469, lrate = 0.010, error = 6.000
>epoch = 

>epoch = 145, lrate = 0.010, error = 26.000
>epoch = 146, lrate = 0.010, error = 30.000
>epoch = 147, lrate = 0.010, error = 30.000
>epoch = 148, lrate = 0.010, error = 25.000
>epoch = 149, lrate = 0.010, error = 36.000
>epoch = 150, lrate = 0.010, error = 32.000
>epoch = 151, lrate = 0.010, error = 30.000
>epoch = 152, lrate = 0.010, error = 25.000
>epoch = 153, lrate = 0.010, error = 29.000
>epoch = 154, lrate = 0.010, error = 32.000
>epoch = 155, lrate = 0.010, error = 36.000
>epoch = 156, lrate = 0.010, error = 38.000
>epoch = 157, lrate = 0.010, error = 24.000
>epoch = 158, lrate = 0.010, error = 30.000
>epoch = 159, lrate = 0.010, error = 24.000
>epoch = 160, lrate = 0.010, error = 28.000
>epoch = 161, lrate = 0.010, error = 32.000
>epoch = 162, lrate = 0.010, error = 28.000
>epoch = 163, lrate = 0.010, error = 32.000
>epoch = 164, lrate = 0.010, error = 26.000
>epoch = 165, lrate = 0.010, error = 22.000
>epoch = 166, lrate = 0.010, error = 34.000
>epoch = 167, lrate = 0.010, err

>epoch = 334, lrate = 0.010, error = 26.000
>epoch = 335, lrate = 0.010, error = 22.000
>epoch = 336, lrate = 0.010, error = 22.000
>epoch = 337, lrate = 0.010, error = 24.000
>epoch = 338, lrate = 0.010, error = 22.000
>epoch = 339, lrate = 0.010, error = 22.000
>epoch = 340, lrate = 0.010, error = 20.000
>epoch = 341, lrate = 0.010, error = 26.000
>epoch = 342, lrate = 0.010, error = 22.000
>epoch = 343, lrate = 0.010, error = 22.000
>epoch = 344, lrate = 0.010, error = 26.000
>epoch = 345, lrate = 0.010, error = 26.000
>epoch = 346, lrate = 0.010, error = 26.000
>epoch = 347, lrate = 0.010, error = 26.000
>epoch = 348, lrate = 0.010, error = 14.000
>epoch = 349, lrate = 0.010, error = 24.000
>epoch = 350, lrate = 0.010, error = 30.000
>epoch = 351, lrate = 0.010, error = 18.000
>epoch = 352, lrate = 0.010, error = 27.000
>epoch = 353, lrate = 0.010, error = 19.000
>epoch = 354, lrate = 0.010, error = 24.000
>epoch = 355, lrate = 0.010, error = 20.000
>epoch = 356, lrate = 0.010, err

>epoch = 30, lrate = 0.010, error = 34.000
>epoch = 31, lrate = 0.010, error = 40.000
>epoch = 32, lrate = 0.010, error = 38.000
>epoch = 33, lrate = 0.010, error = 40.000
>epoch = 34, lrate = 0.010, error = 45.000
>epoch = 35, lrate = 0.010, error = 48.000
>epoch = 36, lrate = 0.010, error = 42.000
>epoch = 37, lrate = 0.010, error = 32.000
>epoch = 38, lrate = 0.010, error = 37.000
>epoch = 39, lrate = 0.010, error = 38.000
>epoch = 40, lrate = 0.010, error = 38.000
>epoch = 41, lrate = 0.010, error = 35.000
>epoch = 42, lrate = 0.010, error = 38.000
>epoch = 43, lrate = 0.010, error = 34.000
>epoch = 44, lrate = 0.010, error = 32.000
>epoch = 45, lrate = 0.010, error = 40.000
>epoch = 46, lrate = 0.010, error = 44.000
>epoch = 47, lrate = 0.010, error = 36.000
>epoch = 48, lrate = 0.010, error = 33.000
>epoch = 49, lrate = 0.010, error = 29.000
>epoch = 50, lrate = 0.010, error = 37.000
>epoch = 51, lrate = 0.010, error = 31.000
>epoch = 52, lrate = 0.010, error = 40.000
>epoch = 53

>epoch = 244, lrate = 0.010, error = 38.000
>epoch = 245, lrate = 0.010, error = 34.000
>epoch = 246, lrate = 0.010, error = 38.000
>epoch = 247, lrate = 0.010, error = 25.000
>epoch = 248, lrate = 0.010, error = 32.000
>epoch = 249, lrate = 0.010, error = 38.000
>epoch = 250, lrate = 0.010, error = 39.000
>epoch = 251, lrate = 0.010, error = 36.000
>epoch = 252, lrate = 0.010, error = 34.000
>epoch = 253, lrate = 0.010, error = 32.000
>epoch = 254, lrate = 0.010, error = 36.000
>epoch = 255, lrate = 0.010, error = 26.000
>epoch = 256, lrate = 0.010, error = 28.000
>epoch = 257, lrate = 0.010, error = 32.000
>epoch = 258, lrate = 0.010, error = 39.000
>epoch = 259, lrate = 0.010, error = 32.000
>epoch = 260, lrate = 0.010, error = 25.000
>epoch = 261, lrate = 0.010, error = 34.000
>epoch = 262, lrate = 0.010, error = 32.000
>epoch = 263, lrate = 0.010, error = 36.000
>epoch = 264, lrate = 0.010, error = 36.000
>epoch = 265, lrate = 0.010, error = 36.000
>epoch = 266, lrate = 0.010, err

              precision    recall  f1-score   support

           0       0.93      0.42      0.58        31
           1       0.67      0.97      0.80        38

    accuracy                           0.72        69
   macro avg       0.80      0.70      0.69        69
weighted avg       0.79      0.72      0.70        69

Scores: [63.76811594202898, 76.81159420289855, 72.46376811594203]
Mean Accuracy: 71.014%



In [39]:
#perceptron on 1000+ dimension dataset
import random
seed(1)
dataset100 = list()
sample = list()
for _ in range(100):
    sample = [random.random() for _ in range(100)]
    if sample[0] >= 0:
        sample.append(1)
    elif sample[0] < 0:
        sample.append(0)
    dataset100.append(sample)

# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))
print()

>epoch = 0, lrate = 0.010, error = 59.000
>epoch = 1, lrate = 0.010, error = 50.000
>epoch = 2, lrate = 0.010, error = 50.000
>epoch = 3, lrate = 0.010, error = 49.000
>epoch = 4, lrate = 0.010, error = 46.000
>epoch = 5, lrate = 0.010, error = 45.000
>epoch = 6, lrate = 0.010, error = 43.000
>epoch = 7, lrate = 0.010, error = 45.000
>epoch = 8, lrate = 0.010, error = 44.000
>epoch = 9, lrate = 0.010, error = 45.000
>epoch = 10, lrate = 0.010, error = 46.000
>epoch = 11, lrate = 0.010, error = 41.000
>epoch = 12, lrate = 0.010, error = 42.000
>epoch = 13, lrate = 0.010, error = 45.000
>epoch = 14, lrate = 0.010, error = 32.000
>epoch = 15, lrate = 0.010, error = 36.000
>epoch = 16, lrate = 0.010, error = 39.000
>epoch = 17, lrate = 0.010, error = 36.000
>epoch = 18, lrate = 0.010, error = 39.000
>epoch = 19, lrate = 0.010, error = 44.000
>epoch = 20, lrate = 0.010, error = 41.000
>epoch = 21, lrate = 0.010, error = 40.000
>epoch = 22, lrate = 0.010, error = 34.000
>epoch = 23, lrate = 

              precision    recall  f1-score   support

           0       0.71      0.78      0.75        32
           1       0.79      0.73      0.76        37

    accuracy                           0.75        69
   macro avg       0.75      0.76      0.75        69
weighted avg       0.76      0.75      0.75        69

>epoch = 0, lrate = 0.010, error = 59.000
>epoch = 1, lrate = 0.010, error = 44.000
>epoch = 2, lrate = 0.010, error = 45.000
>epoch = 3, lrate = 0.010, error = 39.000
>epoch = 4, lrate = 0.010, error = 30.000
>epoch = 5, lrate = 0.010, error = 38.000
>epoch = 6, lrate = 0.010, error = 33.000
>epoch = 7, lrate = 0.010, error = 31.000
>epoch = 8, lrate = 0.010, error = 33.000
>epoch = 9, lrate = 0.010, error = 35.000
>epoch = 10, lrate = 0.010, error = 31.000
>epoch = 11, lrate = 0.010, error = 33.000
>epoch = 12, lrate = 0.010, error = 28.000
>epoch = 13, lrate = 0.010, error = 33.000
>epoch = 14, lrate = 0.010, error = 35.000
>epoch = 15, lrate = 0.010, error = 36

>epoch = 216, lrate = 0.010, error = 13.000
>epoch = 217, lrate = 0.010, error = 23.000
>epoch = 218, lrate = 0.010, error = 13.000
>epoch = 219, lrate = 0.010, error = 21.000
>epoch = 220, lrate = 0.010, error = 15.000
>epoch = 221, lrate = 0.010, error = 24.000
>epoch = 222, lrate = 0.010, error = 23.000
>epoch = 223, lrate = 0.010, error = 13.000
>epoch = 224, lrate = 0.010, error = 12.000
>epoch = 225, lrate = 0.010, error = 12.000
>epoch = 226, lrate = 0.010, error = 17.000
>epoch = 227, lrate = 0.010, error = 15.000
>epoch = 228, lrate = 0.010, error = 16.000
>epoch = 229, lrate = 0.010, error = 21.000
>epoch = 230, lrate = 0.010, error = 18.000
>epoch = 231, lrate = 0.010, error = 20.000
>epoch = 232, lrate = 0.010, error = 25.000
>epoch = 233, lrate = 0.010, error = 15.000
>epoch = 234, lrate = 0.010, error = 21.000
>epoch = 235, lrate = 0.010, error = 17.000
>epoch = 236, lrate = 0.010, error = 17.000
>epoch = 237, lrate = 0.010, error = 7.000
>epoch = 238, lrate = 0.010, erro

              precision    recall  f1-score   support

           0       0.84      0.62      0.71        34
           1       0.70      0.89      0.78        35

    accuracy                           0.75        69
   macro avg       0.77      0.75      0.75        69
weighted avg       0.77      0.75      0.75        69

>epoch = 0, lrate = 0.010, error = 58.000
>epoch = 1, lrate = 0.010, error = 49.000
>epoch = 2, lrate = 0.010, error = 42.000
>epoch = 3, lrate = 0.010, error = 48.000
>epoch = 4, lrate = 0.010, error = 41.000
>epoch = 5, lrate = 0.010, error = 36.000
>epoch = 6, lrate = 0.010, error = 32.000
>epoch = 7, lrate = 0.010, error = 37.000
>epoch = 8, lrate = 0.010, error = 30.000
>epoch = 9, lrate = 0.010, error = 30.000
>epoch = 10, lrate = 0.010, error = 22.000
>epoch = 11, lrate = 0.010, error = 24.000
>epoch = 12, lrate = 0.010, error = 27.000
>epoch = 13, lrate = 0.010, error = 26.000
>epoch = 14, lrate = 0.010, error = 24.000
>epoch = 15, lrate = 0.010, error = 27

>epoch = 417, lrate = 0.010, error = 12.000
>epoch = 418, lrate = 0.010, error = 10.000
>epoch = 419, lrate = 0.010, error = 10.000
>epoch = 420, lrate = 0.010, error = 16.000
>epoch = 421, lrate = 0.010, error = 8.000
>epoch = 422, lrate = 0.010, error = 8.000
>epoch = 423, lrate = 0.010, error = 8.000
>epoch = 424, lrate = 0.010, error = 10.000
>epoch = 425, lrate = 0.010, error = 14.000
>epoch = 426, lrate = 0.010, error = 16.000
>epoch = 427, lrate = 0.010, error = 8.000
>epoch = 428, lrate = 0.010, error = 8.000
>epoch = 429, lrate = 0.010, error = 10.000
>epoch = 430, lrate = 0.010, error = 8.000
>epoch = 431, lrate = 0.010, error = 10.000
>epoch = 432, lrate = 0.010, error = 10.000
>epoch = 433, lrate = 0.010, error = 16.000
>epoch = 434, lrate = 0.010, error = 10.000
>epoch = 435, lrate = 0.010, error = 10.000
>epoch = 436, lrate = 0.010, error = 12.000
>epoch = 437, lrate = 0.010, error = 10.000
>epoch = 438, lrate = 0.010, error = 14.000
>epoch = 439, lrate = 0.010, error = 6

In [40]:
# perceptron on text dataset
import random
seed(1)
datasetText = list()
sample = list()
for _ in range (1000):
    sample = [random.randint(0, 1) for _ in range(10)]
    datasetText.append(sample)

for i in range(len(datasetText)):
    vocab = datasetText[i]
    if (vocab[2] == 1 & vocab[4] == 1 & vocab[8] == 1):
        vocab.append(1)
    else:
        vocab.append(0)
        
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))
print()

>epoch = 0, lrate = 0.010, error = 53.000
>epoch = 1, lrate = 0.010, error = 44.000
>epoch = 2, lrate = 0.010, error = 38.000
>epoch = 3, lrate = 0.010, error = 43.000
>epoch = 4, lrate = 0.010, error = 40.000
>epoch = 5, lrate = 0.010, error = 38.000
>epoch = 6, lrate = 0.010, error = 44.000
>epoch = 7, lrate = 0.010, error = 40.000
>epoch = 8, lrate = 0.010, error = 43.000
>epoch = 9, lrate = 0.010, error = 38.000
>epoch = 10, lrate = 0.010, error = 45.000
>epoch = 11, lrate = 0.010, error = 34.000
>epoch = 12, lrate = 0.010, error = 32.000
>epoch = 13, lrate = 0.010, error = 39.000
>epoch = 14, lrate = 0.010, error = 34.000
>epoch = 15, lrate = 0.010, error = 38.000
>epoch = 16, lrate = 0.010, error = 39.000
>epoch = 17, lrate = 0.010, error = 29.000
>epoch = 18, lrate = 0.010, error = 37.000
>epoch = 19, lrate = 0.010, error = 42.000
>epoch = 20, lrate = 0.010, error = 38.000
>epoch = 21, lrate = 0.010, error = 37.000
>epoch = 22, lrate = 0.010, error = 37.000
>epoch = 23, lrate = 

>epoch = 308, lrate = 0.010, error = 26.000
>epoch = 309, lrate = 0.010, error = 30.000
>epoch = 310, lrate = 0.010, error = 26.000
>epoch = 311, lrate = 0.010, error = 25.000
>epoch = 312, lrate = 0.010, error = 27.000
>epoch = 313, lrate = 0.010, error = 32.000
>epoch = 314, lrate = 0.010, error = 24.000
>epoch = 315, lrate = 0.010, error = 25.000
>epoch = 316, lrate = 0.010, error = 27.000
>epoch = 317, lrate = 0.010, error = 28.000
>epoch = 318, lrate = 0.010, error = 28.000
>epoch = 319, lrate = 0.010, error = 26.000
>epoch = 320, lrate = 0.010, error = 28.000
>epoch = 321, lrate = 0.010, error = 24.000
>epoch = 322, lrate = 0.010, error = 26.000
>epoch = 323, lrate = 0.010, error = 28.000
>epoch = 324, lrate = 0.010, error = 24.000
>epoch = 325, lrate = 0.010, error = 30.000
>epoch = 326, lrate = 0.010, error = 28.000
>epoch = 327, lrate = 0.010, error = 28.000
>epoch = 328, lrate = 0.010, error = 28.000
>epoch = 329, lrate = 0.010, error = 24.000
>epoch = 330, lrate = 0.010, err

>epoch = 123, lrate = 0.010, error = 28.000
>epoch = 124, lrate = 0.010, error = 27.000
>epoch = 125, lrate = 0.010, error = 31.000
>epoch = 126, lrate = 0.010, error = 29.000
>epoch = 127, lrate = 0.010, error = 32.000
>epoch = 128, lrate = 0.010, error = 29.000
>epoch = 129, lrate = 0.010, error = 25.000
>epoch = 130, lrate = 0.010, error = 30.000
>epoch = 131, lrate = 0.010, error = 34.000
>epoch = 132, lrate = 0.010, error = 31.000
>epoch = 133, lrate = 0.010, error = 27.000
>epoch = 134, lrate = 0.010, error = 24.000
>epoch = 135, lrate = 0.010, error = 29.000
>epoch = 136, lrate = 0.010, error = 30.000
>epoch = 137, lrate = 0.010, error = 27.000
>epoch = 138, lrate = 0.010, error = 25.000
>epoch = 139, lrate = 0.010, error = 26.000
>epoch = 140, lrate = 0.010, error = 25.000
>epoch = 141, lrate = 0.010, error = 25.000
>epoch = 142, lrate = 0.010, error = 28.000
>epoch = 143, lrate = 0.010, error = 31.000
>epoch = 144, lrate = 0.010, error = 29.000
>epoch = 145, lrate = 0.010, err

              precision    recall  f1-score   support

           0       0.77      0.77      0.77        26
           1       0.86      0.86      0.86        43

    accuracy                           0.83        69
   macro avg       0.81      0.81      0.81        69
weighted avg       0.83      0.83      0.83        69

>epoch = 0, lrate = 0.010, error = 56.000
>epoch = 1, lrate = 0.010, error = 47.000
>epoch = 2, lrate = 0.010, error = 41.000
>epoch = 3, lrate = 0.010, error = 39.000
>epoch = 4, lrate = 0.010, error = 43.000
>epoch = 5, lrate = 0.010, error = 35.000
>epoch = 6, lrate = 0.010, error = 37.000
>epoch = 7, lrate = 0.010, error = 35.000
>epoch = 8, lrate = 0.010, error = 32.000
>epoch = 9, lrate = 0.010, error = 28.000
>epoch = 10, lrate = 0.010, error = 36.000
>epoch = 11, lrate = 0.010, error = 37.000
>epoch = 12, lrate = 0.010, error = 26.000
>epoch = 13, lrate = 0.010, error = 34.000
>epoch = 14, lrate = 0.010, error = 27.000
>epoch = 15, lrate = 0.010, error = 28

              precision    recall  f1-score   support

           0       0.83      0.69      0.75        35
           1       0.72      0.85      0.78        34

    accuracy                           0.77        69
   macro avg       0.78      0.77      0.77        69
weighted avg       0.78      0.77      0.77        69

Scores: [68.11594202898551, 82.6086956521739, 76.81159420289855]
Mean Accuracy: 75.845%



## Problem 4
Problem 4.1 is done by hand on a sheet of paper and it has been included in the zip file

In [41]:
# Test the Perceptron algorithm on the small dataset
seed(0)
# load and prepare data
filename = 'small_dataset.csv'
datasetSmall = load_csv(filename)
for i in range(len(datasetSmall[0]) - 1):
    str_column_to_float(datasetSmall, i)
# convert string class to integers
str_column_to_int(datasetSmall, len(datasetSmall[0]) - 1)

{'-1': 0, '1': 1}

In [42]:
sample = [0.5, 0.6, 0]
datasetSmall.append(sample)
print(datasetSmall)
print()

n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(datasetSmall, perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))
print()

[[0.3, 1.0, 0], [0.7, 0.5, 0], [0.3, 0.6, 0], [0.8, 0.8, 0], [0.7, 0.6, 0], [0.8, 0.6, 0], [0.9, 1.0, 0], [0.3, 0.3, 0], [0.5, 0.9, 0], [0.9, 0.3, 0], [1.3, 1.2, 1], [1.6, 1.9, 1], [1.3, 1.5, 1], [1.1, 1.8, 1], [2.0, 1.5, 1], [1.3, 1.1, 1], [1.7, 1.8, 1], [1.8, 1.7, 1], [1.9, 1.7, 1], [1.4, 1.7, 1], [0.5, 0.6, 0]]

>epoch = 0, lrate = 0.010, error = 4.000
>epoch = 1, lrate = 0.010, error = 6.000
>epoch = 2, lrate = 0.010, error = 2.000
>epoch = 3, lrate = 0.010, error = 1.000
>epoch = 4, lrate = 0.010, error = 0.000
>epoch = 5, lrate = 0.010, error = 0.000
>epoch = 6, lrate = 0.010, error = 0.000
>epoch = 7, lrate = 0.010, error = 0.000
>epoch = 8, lrate = 0.010, error = 0.000
>epoch = 9, lrate = 0.010, error = 0.000
>epoch = 10, lrate = 0.010, error = 0.000
>epoch = 11, lrate = 0.010, error = 0.000
>epoch = 12, lrate = 0.010, error = 0.000
>epoch = 13, lrate = 0.010, error = 0.000
>epoch = 14, lrate = 0.010, error = 0.000
>epoch = 15, lrate = 0.010, error = 0.000
>epoch = 16, lrate = 

>epoch = 21, lrate = 0.010, error = 0.000
>epoch = 22, lrate = 0.010, error = 0.000
>epoch = 23, lrate = 0.010, error = 0.000
>epoch = 24, lrate = 0.010, error = 0.000
>epoch = 25, lrate = 0.010, error = 0.000
>epoch = 26, lrate = 0.010, error = 0.000
>epoch = 27, lrate = 0.010, error = 0.000
>epoch = 28, lrate = 0.010, error = 0.000
>epoch = 29, lrate = 0.010, error = 0.000
>epoch = 30, lrate = 0.010, error = 0.000
>epoch = 31, lrate = 0.010, error = 0.000
>epoch = 32, lrate = 0.010, error = 0.000
>epoch = 33, lrate = 0.010, error = 0.000
>epoch = 34, lrate = 0.010, error = 0.000
>epoch = 35, lrate = 0.010, error = 0.000
>epoch = 36, lrate = 0.010, error = 0.000
>epoch = 37, lrate = 0.010, error = 0.000
>epoch = 38, lrate = 0.010, error = 0.000
>epoch = 39, lrate = 0.010, error = 0.000
>epoch = 40, lrate = 0.010, error = 0.000
>epoch = 41, lrate = 0.010, error = 0.000
>epoch = 42, lrate = 0.010, error = 0.000
>epoch = 43, lrate = 0.010, error = 0.000
>epoch = 44, lrate = 0.010, error 

>epoch = 240, lrate = 0.010, error = 0.000
>epoch = 241, lrate = 0.010, error = 0.000
>epoch = 242, lrate = 0.010, error = 0.000
>epoch = 243, lrate = 0.010, error = 0.000
>epoch = 244, lrate = 0.010, error = 0.000
>epoch = 245, lrate = 0.010, error = 0.000
>epoch = 246, lrate = 0.010, error = 0.000
>epoch = 247, lrate = 0.010, error = 0.000
>epoch = 248, lrate = 0.010, error = 0.000
>epoch = 249, lrate = 0.010, error = 0.000
>epoch = 250, lrate = 0.010, error = 0.000
>epoch = 251, lrate = 0.010, error = 0.000
>epoch = 252, lrate = 0.010, error = 0.000
>epoch = 253, lrate = 0.010, error = 0.000
>epoch = 254, lrate = 0.010, error = 0.000
>epoch = 255, lrate = 0.010, error = 0.000
>epoch = 256, lrate = 0.010, error = 0.000
>epoch = 257, lrate = 0.010, error = 0.000
>epoch = 258, lrate = 0.010, error = 0.000
>epoch = 259, lrate = 0.010, error = 0.000
>epoch = 260, lrate = 0.010, error = 0.000
>epoch = 261, lrate = 0.010, error = 0.000
>epoch = 262, lrate = 0.010, error = 0.000
>epoch = 26

## Problem 5

After comparing all the learning rates from 0.01 to 0.10 through a 0.01 interval, the learning rate of 0.02 produced the best results with the highest accuracy and high precision, recall, and f1 scores

## Problem 6

After testing different learning rates from 0.01 to 0.10, there was not a noticeable difference in runtime, all the rates were processed quickly

## Problem 7

In the code below the learning rate decreases by 0.5% everytime the error rate is found to be increased when compared to the previous iteration

In [43]:
# Estimate Perceptron weights using stochastic gradient descent
def error_train_weights(train, l_rate, n_epoch):
    error_tracker = 0
    weights = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0.0
        for row in train:
            prediction = predict(row, weights)
            error = row[-1] - prediction
            if error > error_tracker:
                l_rate -= l_rate * 0.005
            sum_error += error**2
            weights[0] = weights[0] + l_rate * error
            for i in range(len(row) - 1):
                weights[i+1] = weights[i + 1] + l_rate * error * row[i]
        print('>epoch = %d, lrate = %.10f, error = %.3f' % (epoch, l_rate, sum_error))
    return weights

In [44]:
# Perceptron Algorithm With Stochastic Gradient Descent
def new_perceptron(train, test, l_rate, n_epoch):
    predictions = list()
    weights = error_train_weights(train, l_rate, n_epoch)
    for row in test:
        prediction = predict(row, weights)
        predictions.append(prediction)
    return(predictions)

In [45]:
# evaluate algorithm
n_folds = 3
l_rate = 0.01
n_epoch = 500
scores = evaluate_algorithm(dataset, new_perceptron, n_folds, l_rate, n_epoch)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

>epoch = 0, lrate = 0.0086905300, error = 57.000
>epoch = 1, lrate = 0.0077442099, error = 47.000
>epoch = 2, lrate = 0.0069704660, error = 42.000
>epoch = 3, lrate = 0.0063690883, error = 37.000
>epoch = 4, lrate = 0.0058195944, error = 37.000
>epoch = 5, lrate = 0.0054252012, error = 28.000
>epoch = 6, lrate = 0.0049820516, error = 34.000
>epoch = 7, lrate = 0.0047384798, error = 21.000
>epoch = 8, lrate = 0.0044395513, error = 27.000
>epoch = 9, lrate = 0.0041594809, error = 26.000
>epoch = 10, lrate = 0.0038970787, error = 27.000
>epoch = 11, lrate = 0.0036695782, error = 24.000
>epoch = 12, lrate = 0.0034208913, error = 28.000
>epoch = 13, lrate = 0.0032211893, error = 25.000
>epoch = 14, lrate = 0.0030637058, error = 20.000
>epoch = 15, lrate = 0.0028704310, error = 27.000
>epoch = 16, lrate = 0.0027028633, error = 24.000
>epoch = 17, lrate = 0.0025196905, error = 28.000
>epoch = 18, lrate = 0.0024085459, error = 19.000
>epoch = 19, lrate = 0.0022793385, error = 21.000
>epoch = 2

>epoch = 346, lrate = 0.0000000220, error = 12.000
>epoch = 347, lrate = 0.0000000212, error = 13.000
>epoch = 348, lrate = 0.0000000206, error = 12.000
>epoch = 349, lrate = 0.0000000200, error = 12.000
>epoch = 350, lrate = 0.0000000194, error = 12.000
>epoch = 351, lrate = 0.0000000188, error = 12.000
>epoch = 352, lrate = 0.0000000182, error = 13.000
>epoch = 353, lrate = 0.0000000176, error = 12.000
>epoch = 354, lrate = 0.0000000171, error = 12.000
>epoch = 355, lrate = 0.0000000166, error = 12.000
>epoch = 356, lrate = 0.0000000161, error = 12.000
>epoch = 357, lrate = 0.0000000156, error = 12.000
>epoch = 358, lrate = 0.0000000151, error = 13.000
>epoch = 359, lrate = 0.0000000147, error = 12.000
>epoch = 360, lrate = 0.0000000143, error = 11.000
>epoch = 361, lrate = 0.0000000138, error = 13.000
>epoch = 362, lrate = 0.0000000134, error = 12.000
>epoch = 363, lrate = 0.0000000130, error = 12.000
>epoch = 364, lrate = 0.0000000125, error = 13.000
>epoch = 365, lrate = 0.0000000

>epoch = 172, lrate = 0.0000010381, error = 19.000
>epoch = 173, lrate = 0.0000009923, error = 18.000
>epoch = 174, lrate = 0.0000009438, error = 20.000
>epoch = 175, lrate = 0.0000008977, error = 19.000
>epoch = 176, lrate = 0.0000008538, error = 20.000
>epoch = 177, lrate = 0.0000008161, error = 18.000
>epoch = 178, lrate = 0.0000007762, error = 19.000
>epoch = 179, lrate = 0.0000007383, error = 20.000
>epoch = 180, lrate = 0.0000007057, error = 18.000
>epoch = 181, lrate = 0.0000006746, error = 18.000
>epoch = 182, lrate = 0.0000006416, error = 19.000
>epoch = 183, lrate = 0.0000006102, error = 20.000
>epoch = 184, lrate = 0.0000005833, error = 18.000
>epoch = 185, lrate = 0.0000005548, error = 19.000
>epoch = 186, lrate = 0.0000005277, error = 20.000
>epoch = 187, lrate = 0.0000005044, error = 18.000
>epoch = 188, lrate = 0.0000004797, error = 20.000
>epoch = 189, lrate = 0.0000004563, error = 19.000
>epoch = 190, lrate = 0.0000004340, error = 20.000
>epoch = 191, lrate = 0.0000004

>epoch = 359, lrate = 0.0000000001, error = 18.000
>epoch = 360, lrate = 0.0000000001, error = 18.000
>epoch = 361, lrate = 0.0000000001, error = 19.000
>epoch = 362, lrate = 0.0000000001, error = 20.000
>epoch = 363, lrate = 0.0000000001, error = 18.000
>epoch = 364, lrate = 0.0000000001, error = 20.000
>epoch = 365, lrate = 0.0000000001, error = 19.000
>epoch = 366, lrate = 0.0000000001, error = 18.000
>epoch = 367, lrate = 0.0000000001, error = 18.000
>epoch = 368, lrate = 0.0000000001, error = 20.000
>epoch = 369, lrate = 0.0000000001, error = 19.000
>epoch = 370, lrate = 0.0000000001, error = 18.000
>epoch = 371, lrate = 0.0000000001, error = 20.000
>epoch = 372, lrate = 0.0000000001, error = 19.000
>epoch = 373, lrate = 0.0000000001, error = 20.000
>epoch = 374, lrate = 0.0000000001, error = 18.000
>epoch = 375, lrate = 0.0000000001, error = 19.000
>epoch = 376, lrate = 0.0000000001, error = 20.000
>epoch = 377, lrate = 0.0000000000, error = 18.000
>epoch = 378, lrate = 0.0000000

>epoch = 297, lrate = 0.0000000191, error = 16.000
>epoch = 298, lrate = 0.0000000184, error = 16.000
>epoch = 299, lrate = 0.0000000176, error = 16.000
>epoch = 300, lrate = 0.0000000170, error = 15.000
>epoch = 301, lrate = 0.0000000164, error = 15.000
>epoch = 302, lrate = 0.0000000157, error = 16.000
>epoch = 303, lrate = 0.0000000151, error = 16.000
>epoch = 304, lrate = 0.0000000145, error = 16.000
>epoch = 305, lrate = 0.0000000140, error = 15.000
>epoch = 306, lrate = 0.0000000135, error = 15.000
>epoch = 307, lrate = 0.0000000129, error = 16.000
>epoch = 308, lrate = 0.0000000124, error = 16.000
>epoch = 309, lrate = 0.0000000119, error = 16.000
>epoch = 310, lrate = 0.0000000115, error = 15.000
>epoch = 311, lrate = 0.0000000111, error = 15.000
>epoch = 312, lrate = 0.0000000107, error = 14.000
>epoch = 313, lrate = 0.0000000103, error = 14.000
>epoch = 314, lrate = 0.0000000099, error = 16.000
>epoch = 315, lrate = 0.0000000096, error = 14.000
>epoch = 316, lrate = 0.0000000

>epoch = 480, lrate = 0.0000000000, error = 15.000
>epoch = 481, lrate = 0.0000000000, error = 15.000
>epoch = 482, lrate = 0.0000000000, error = 15.000
>epoch = 483, lrate = 0.0000000000, error = 16.000
>epoch = 484, lrate = 0.0000000000, error = 16.000
>epoch = 485, lrate = 0.0000000000, error = 16.000
>epoch = 486, lrate = 0.0000000000, error = 16.000
>epoch = 487, lrate = 0.0000000000, error = 16.000
>epoch = 488, lrate = 0.0000000000, error = 16.000
>epoch = 489, lrate = 0.0000000000, error = 15.000
>epoch = 490, lrate = 0.0000000000, error = 15.000
>epoch = 491, lrate = 0.0000000000, error = 16.000
>epoch = 492, lrate = 0.0000000000, error = 16.000
>epoch = 493, lrate = 0.0000000000, error = 16.000
>epoch = 494, lrate = 0.0000000000, error = 15.000
>epoch = 495, lrate = 0.0000000000, error = 15.000
>epoch = 496, lrate = 0.0000000000, error = 16.000
>epoch = 497, lrate = 0.0000000000, error = 14.000
>epoch = 498, lrate = 0.0000000000, error = 14.000
>epoch = 499, lrate = 0.0000000

## Problem 8

We can use the naive bayes algorithm to perform classification. Using the coordinates (features) of the data points to classify each of the data points into specific classes (labels). Since the data is already linearly separable there are easy features to find to accurately classify the data into two different classes. Another ML algorithm we can use is k-Means, and that is because the data points would be in two clusters since they are linearly separable. Therefore, two centroids could be made to classify the data points into two clusters (labels).